In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.0.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0 pyspark-shell'

In [15]:
!pip install pyspark=="3.0.0"

  Using cached pyspark-3.0.0-py2.py3-none-any.whl
  Attempting uninstall: pyspark
    Found existing installation: pyspark 3.5.0
    Can't uninstall 'pyspark'. No files were found to uninstall.


In [6]:
import pyspark

In [14]:
!pip uninstall pyspark

Found existing installation: pyspark 3.5.0
Can't uninstall 'pyspark'. No files were found to uninstall.


In [13]:
 pyspark.__version__

'3.5.0'

In [2]:
import pyspark
from pyspark.sql import SparkSession

In [3]:

from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json
from pyspark.sql.types import StructType, StringType, DoubleType, LongType

spark = SparkSession.builder \
    .appName("KafkaReader") \
    .getOrCreate()

schema_rates = StructType() \
    .add("id", StringType()) \
    .add("symbol", StringType()) \
    .add("currencySymbol", StringType()) \
    .add("type", StringType()) \
    .add("rateUsd", DoubleType()) \
    .add("timestamp", LongType())

rates_df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "10.0.0.10:9092") \
    .option("subscribe", "crypto_rates") \
    .option("startingOffsets", "latest") \
    .option("failOnDataLoss", "false") \
    .load() \
    .selectExpr("CAST(value AS STRING)") \
    .select(from_json("value", schema_rates).alias("data")) \
    .select("data.*")

query = rates_df.writeStream \
    .format("json") \
    .outputMode("append") \
    .option("path", "output_json") \
    .option("checkpointLocation", "checkpoint_json") \
    .start()

spark.streams.awaitAnyTermination()

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: reentrant call inside <_io.BufferedReader name=52>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip

Py4JError: An error occurred while calling o55.awaitAnyTermination

In [3]:
from pyspark.sql import functions as F
spark2 = SparkSession.builder \
    .appName("KafkaReadeandWrite") \
    .getOrCreate()

rates_df = spark2 \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "10.0.0.10:9092") \
    .option("subscribe", "crypto_rates") \
    .option("startingOffsets", "earliest") \
    .load() 

query = rates_df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")\
    .writeStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "10.0.0.10:9092") \
    .option("topic", "topic_modified") \
    .option("checkpointLocation", "checkpoint_rate_modified") \
    .start()

query.awaitTermination()

StreamingQueryException: [STREAM_FAILED] Query [id = c78b5e8b-86b2-4f17-96a3-8969b9edf008, runId = 1a7127e4-f034-4dc2-a824-9ac6659a4fc8] terminated with exception: org/apache/spark/sql/internal/connector/SupportsStreamingUpdate

In [4]:
spark2 = SparkSession.builder \
    .appName("KafkaReadeandWrite") \
    .getOrCreate()

rates_df = spark2 \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "10.0.0.10:9092") \
    .option("subscribe", "crypto_rates") \
    .option("startingOffsets", "earliest") \
    .load() 

query = rates_df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")\
    .writeStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "10.0.0.10:9092") \
    .option("topic", "topic_modified") \
    .option("checkpointLocation", "checkpoint_rate_modified") \
    .start()

query.awaitTermination()

'3.5.0'

In [ ]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.0.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0 pyspark-shell'
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json
from pyspark.sql.types import StructType, StringType, DoubleType, LongType
spark = SparkSession.builder \
    .appName("KafkaReadeandWrite") \
    .getOrCreate()

rates_df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "10.0.0.10:9092") \
    .option("subscribe", "crypto_rates") \
    .option("startingOffsets", "earliest") \
    .load() 

query = rates_df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")\
    .writeStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "10.0.0.10:9092") \
    .option("topic", "topic_modified") \
    .option("checkpointLocation", "checkpoint_rate_modified") \
    .start()

query.awaitTermination()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json
from pyspark.sql.types import StructType, StringType, DoubleType, LongType
spark = SparkSession.builder \
    .appName("KafkaReadeandWrite") \
    .getOrCreate()

rates_df = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "10.0.0.10:9092") \
    .option("subscribe", "crypto_rates") \
    .option("startingOffsets", "earliest") \
    .load() 

query = rates_df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")\
    .writeStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "10.0.0.10:9092") \
    .option("topic", "topic_modified") \
    .option("checkpointLocation", "checkpoint_rate_modified") \
    .start()

query.awaitTermination()